In [2]:
import os
import pickle
import sys
import h5py
import torch
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F
from torch.autograd import Variable
import cv2
import numpy as np
from tqdm import tnrange, tqdm_notebook
import models
import sklearn.metrics as metrics
import matplotlib.pyplot as plt
import h5py
from PIL import Image




In [3]:
device = 'cuda:0' # device where you put your data and models
data_path = './' # the path of the 'npc_v4_data.h5' file
batch_size = 16 # the batch size of the data loader
insp_layer = 'conv3' # the middle layer extracted from alexnet, available in {'conv1', 'conv2', 'conv3', 'conv4', 'conv5'}


In [4]:
root_dir = 'C:/Users/admin/Desktop/pytorch_ovo/data/0_presented_images_800/'
resolution = 300
image_path = os.listdir(root_dir)
path_dict = {}
for j in image_path:
    key = int(j.split('_')[0])  # 刺激呈现的顺序是图像名称下划线前面的数字顺序。
    path_dict[key] = j

stim_arr = np.zeros((len(image_path), resolution, resolution, 3))
# stim_arr_gray3 = np.zeros((len(image_path), resolution, resolution, 3))
for i in range(len(image_path)):
    img_bgr = cv2.imread(os.path.join(root_dir, path_dict[i+1]))
    stim_arr[i] = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
stim_arr = stim_arr.astype('float32')

In [5]:
# 我们数据包括880张图片，前800张是unique的，后80张是random出的图片序号，然后这些序号对应图片进行重复播放，作为验证集。
id = h5py.File('C:/Users/admin/Desktop/pytorch_ovo/data/5_L86LL_V4_S10_D270_objects/stimuli/Random_id_80_2021_12_22.mat', 'r')

images_n  = np.zeros(shape=(stim_arr.shape[0], 299, 299, 3))
for i in range(stim_arr.shape[0]):
    images_n[i] = cv2.resize(stim_arr[i], (299, 299))

idx = np.array(id['sampleidlist21']).squeeze().astype('int') - 1
print(idx)
idx, unique_idx = np.unique(idx, return_index=True)
print(idx, unique_idx, images_n.shape)

[  4  15  16  18  19  40  42  54  61  78 120 124 126 133 142 146 161 165
 166 173 174 178 212 215 222 224 262 278 285 293 305 311 336 351 352 364
 368 369 393 400 410 412 420 432 442 463 473 483 484 502 509 510 534 541
 552 579 582 599 604 605 610 615 620 622 633 637 672 675 681 683 692 698
 701 705 711 722 723 727 787 788]
[  4  15  16  18  19  40  42  54  61  78 120 124 126 133 142 146 161 165
 166 173 174 178 212 215 222 224 262 278 285 293 305 311 336 351 352 364
 368 369 393 400 410 412 420 432 442 463 473 483 484 502 509 510 534 541
 552 579 582 599 604 605 610 615 620 622 633 637 672 675 681 683 692 698
 701 705 711 722 723 727 787 788] [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79] (800, 299, 299, 3)


In [6]:
mat_file = h5py.File('C:/Users/admin/Desktop/pytorch_ovo/data/5_L86LL_V4_S10_D270_objects/stimuli/celldataS_110_CalmAn_10_Objects_12_800_80_56_68_74_4_trial_mean_normal.mat', 'r')
#[num_repetitions, num_images, num_neurons]
#print(np.array(mat_file['celldataS']).shape)
neural_n = np.transpose(np.array(mat_file['celldataS']), (1, 2, 0)).astype('float16')
neural_n = neural_n[:,:880, :]
print(neural_n.shape)
#12个trials 880张图片（其中80张是重复），114个细胞

(12, 880, 120)


In [7]:
n_images = 800
n_neurons = neural_n.shape[2]
size_imags = images_n.shape[0]
print(n_images, n_neurons, images_n.shape)

800 120 (800, 299, 299, 3)


In [8]:
#划分训练集和验证集，找到前800张里不重复的作为训练集，取前800张random的和后80张作为验证集，多个trials取平均值
reps = neural_n.shape[0] # trials
rand_ind = np.arange(reps)
np.random.shuffle(rand_ind)

data_y_train = np.concatenate((np.delete(neural_n[:, :800, :], idx, 1), neural_n[:, 880:, :]), 1).mean(0)
temp = np.transpose(neural_n, (1, 0, 2))
print(temp.shape, idx.shape, temp[idx].shape)
data_y_val = np.concatenate((temp[idx], temp[800:880][unique_idx]), 1)
data_y_val = np.transpose(data_y_val, (1, 0, 2))
data_y_val = np.mean(data_y_val, 0)
print(data_y_train.shape)
print(data_y_val.shape)

#
# data_x = images_n[:, np.newaxis].astype(np.float16)
# print('images_n', images_n.shape)
# data_x = data_x / 255 # (640, 1, 299, 299)
# data_x = np.tile(data_x, [1, 3, 1, 1])
# print('data_x', data_x.shape)
# data_x_train = data_x[:576]
# data_x_val = data_x[576:]as indices must be

(880, 12, 120) (80,) (80, 12, 120)
(720, 120)
(80, 120)


In [9]:
print(images_n.shape)
#data_x = images_n[:, np.newaxis].astype(np.float16)
data_x = images_n.astype(np.float16)
print(data_x.shape)
data_x = data_x / 255 # (800, 1, 299, 299)

#data_x = np.tile(data_x, [1, 3, 1, 1])
data_x_train = np.delete(images_n, idx, 0)
data_x_val = images_n[idx]

data_x = np.transpose(data_x, (0, 3, 1, 2))
data_x_train = np.transpose(data_x_train, (0, 3, 1, 2))
data_x_val = np.transpose(data_x_val, (0, 3, 1, 2))
print(data_x.shape, data_x_train.shape, data_x_val.shape)


(800, 299, 299, 3)
(800, 299, 299, 3)
(800, 3, 299, 299) (720, 3, 299, 299) (80, 3, 299, 299)


In [10]:
#设置dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, data_x, data_y):
        self.data_x = data_x
        self.data_y = data_y
    def __getitem__(self, index):
        return index, self.data_x[index], self.data_y[index]
    def __len__(self):
        return self.data_x.shape[0]



dataset_train = Dataset(data_x_train, data_y_train)
dataset_val = Dataset(data_x_val, data_y_val)

loader_train = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, shuffle = True)
loader_val = torch.utils.data.DataLoader(dataset_val, batch_size=batch_size, shuffle = True)

print(f'val: {data_x_val.shape}, {data_y_val.shape}')
# for i,(x,y) in enumerate(loader_val):
#     print(i, x.shape, y.shape)

val: (80, 3, 299, 299), (80, 120)


In [11]:
# CHOOSE THE AUGMENTS IF NECESSARY
lamd_s, lamd_d = [5e-3, 2e-3] # the coefficients of the losses. Try other coefficients!
epoches = 10 # total epochs for training the encoder
lr = 1e-3 # the learing rate for training the encoder

alexnet = models.alexnet(pretrained=True)

#
alexnet.to(device)
alexnet.eval()
for param in alexnet.parameters():
    param.requires_grad_(False)

x = torch.from_numpy(data_x[0:1]).to(device)
print("x:", x.shape)
x = x.float()
fmap = alexnet(x, layer=insp_layer)

neurons = data_y_train.shape[1]
sizes = fmap.shape[2:]
print("fmap: ", fmap.shape)
print("size: ", sizes)
channels = fmap.shape[1]
print(neurons, sizes)
w_s = nn.Parameter(torch.randn(size=(neurons,) + sizes))
print(w_s.shape)


x: torch.Size([1, 3, 299, 299])
fmap:  torch.Size([1, 384, 17, 17])
size:  torch.Size([17, 17])
120 torch.Size([17, 17])
torch.Size([120, 17, 17])


In [12]:
# 把每张图片的conv1 feature map 存起来 理论上会快一点？
x = torch.from_numpy(data_x[0:1]).float().to(device)
fmap = alexnet(x, layer=insp_layer)
print(fmap.shape)
sizes = fmap.shape[2:]

imagenet_mean = torch.tensor([0.485, 0.456, 0.406], dtype=torch.float32).view(1, 3, 1, 1).to(device)
imagenet_std = torch.tensor([0.229, 0.224, 0.225], dtype=torch.float32).view(1, 3, 1, 1).to(device)
transform = lambda x : (x - imagenet_mean) / imagenet_std
#data_x = transform(data_x)
feature_map = torch.Tensor(n_images, fmap.shape[1], fmap.shape[2], fmap.shape[3])
feature_map.to(device)
print(feature_map.shape)
for i in range(n_images):
    x = torch.from_numpy(data_x[i:i + 1]).float().to(device)
    #x = transform(x).to(device)
    fmap = alexnet(x, layer = insp_layer)
    feature_map[i] = fmap

torch.Size([1, 384, 17, 17])
torch.Size([800, 384, 17, 17])


In [13]:
mse_weight = 1.0
l1_weight = 0
spa_weight = 1e-1
ch_weight = 1e-1
lap_weight = 1e-1

In [14]:
#定义损失函数
def mse_loss(prediction, response, weight=None):
    if weight is None:
        mse_loss = torch.mean(torch.mean((prediction - response)**2, dim=1))
    else:
        mse_loss = torch.sum(weight*torch.mean((prediction - response)**2, dim=1))
    return mse_loss

def l2_norm_regularizer(W):
    with torch.autograd.profiler.record_function('l2_norm'):
        penalty = torch.mean(torch.sum(W**2))
        return penalty

def l1_norm_regularizer(W):
    with torch.autograd.profiler.record_function('l1_norm'):
        penalty = torch.mean(torch.sum(torch.abs(W)))
        return penalty

def smoothness_regularizer_2d(W_s):
    K = torch.tensor([
    [0,-1,0],
    [-1,4,-1],
    [0,-1,0]],dtype=torch.float).to(device)
    return torch.sum(F.conv2d(torch.unsqueeze(W_s,1),K.unsqueeze(0).unsqueeze(0))**2)

def pearson_corr(prediction, response):
    prediction = torch.transpose(prediction, 1, 0)
    response = torch.transpose(response, 1, 0)

    prediction_mean = torch.mean(prediction, dim=0)
    response_mean = torch.mean(response, dim=0)

    num = torch.sum((prediction - prediction_mean)*(response - response_mean), dim=0)
    den = torch.sqrt(torch.sum((prediction - prediction_mean)**2, dim=0) *
                     torch.sum((response - response_mean)**2, dim=0))
    pcc = torch.mean(num * (1 / den))
    return pcc

def explained_variance(y_true, y_pred):
    total_var = torch.var(y_true)
    residual_var = torch.var(y_true - y_pred)
    explained_var = total_var - residual_var
    return explained_var.item()


In [15]:
# 模型
class conv_encoder(nn.Module):

    def __init__(self, neurons, sizes, channels):
        super(conv_encoder, self).__init__()
        # PUT YOUR CODES HERE
        self.W_s = nn.Parameter(torch.randn(size=(neurons,) + sizes)) #感受野
        self.W_d = nn.Parameter(torch.randn(size = (neurons,channels,1,1))) #通道权重
        self.W_b = nn.Parameter(torch.randn(size = (1,neurons))) # 偏置


    def forward(self, x):
        # PUT YOUR CODES HERE
        out = torch.einsum('bchw , nhw -> bnchw',x,self.W_s) # dimension : N,n,C,h,w
        out = torch.stack(
            [F.conv2d(out[:,n,:,:,:],torch.unsqueeze(self.W_d[n],0)) for n in range(neurons)],dim=1)
            #dimension:N,n,1,h,w
        out = torch.sum(out,dim=(2,3,4))
        out = out + self.W_b
        return out

def Loss(y, pred, W_s, W_d):
    return mse_loss(y, pred) * mse_weight + \
          l2_norm_regularizer(W_s) * spa_weight + \
          smoothness_regularizer_2d(W_s) * lap_weight + \
          l2_norm_regularizer(W_d) * ch_weight




#encoder = conv_encoder(neurons, sizes, channels).to(device)
encoder = conv_encoder(neurons, sizes, channels).to(device)


In [16]:
def train_model(encoder, optimizer):
    losses = []
    encoder.train()
    for i,(z, x,y) in enumerate(loader_train):
        optimizer.zero_grad()
        x = x.float().to(device)
        y = y.float().to(device)
        z = z.to(device)
        #x = transform(x)
        fmap = feature_map[z - 1].to(device) # 得到z图片id对应的特征图
        out = encoder(fmap) #输出结果
#         print(f'L_e = {l_e} , L_2 = {l_2} , L_l = {l_l}')
        loss = Loss(y, out, encoder.W_s, encoder.W_d)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
#         print(f'iteration {i}, train loss: {losses[-1]}')

    return losses

def validate_model(encoder):
    encoder.eval()
    y_pred = []
    y_true = []
    losses = []
    for i,(z, x,y) in enumerate(loader_val):
        x = x.float().to(device)
        y = y.float().to(device)
        z = z.to(device)
        #x = transform(x)
        fmap = feature_map[z - 1].to(device)
        out = encoder(fmap)
        y_pred.append(out)
        y_true.append(y)
        loss = Loss(y, out, encoder.W_s, encoder.W_d)
        losses.append(loss.item())
    y_pred = torch.cat(y_pred)
    y_true = torch.cat(y_true)
    ev = explained_variance(y_true, y_pred)
    pcc = pearson_corr(y_pred, y_true)
    return pcc, ev,sum(losses)/len(losses)
    #return explained_variance,sum(losses)/len(losses)

"""
    You need to define the conv_encoder() class and train the encoder.
    The code of alexnet has been slightly modified from the torchvision, for convenience
    of extracting the middle layers.

    Example:
        >>> x = x.to(device) # x is a batch of images
        >>> x = transform(x)
        >>> fmap = alexnet(x, layer=insp_layer)
        >>> out= encoder(fmap)
        >>> ...
"""

'\n    You need to define the conv_encoder() class and train the encoder.\n    The code of alexnet has been slightly modified from the torchvision, for convenience\n    of extracting the middle layers.\n\n    Example:\n        >>> x = x.to(device) # x is a batch of images\n        >>> x = transform(x)\n        >>> fmap = alexnet(x, layer=insp_layer)\n        >>> out= encoder(fmap)\n        >>> ...\n'

In [17]:
# losses_train = []
# losses_val = []
# EVs = []

losses_train = []
losses_val = []
EVs = []
pccs = []

In [18]:
lr = 1e-2
optimizer = torch.optim.Adam(encoder.parameters(), lr=lr)
#optimizer = torch.optim.SGD(encoder.parameters(), lr=0.1)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=200, gamma=0.1)

In [19]:
epoches = 2000
best_loss = 1e100
not_improve = 0
endure = 10
for epoch in tqdm_notebook(range(epoches)):
    losses_train += train_model(encoder,optimizer)
    pcc, ev,loss = validate_model(encoder)
    #ev,loss = validate_model(encoder)
    EVs.append(ev)
    pccs.append(pcc)
    losses_val.append(loss)
    train_loss = sum(losses_train[-10:])/10
    if train_loss < best_loss - 1e-5:
        not_improve = 0
    else:
        not_improve += 1
    if epoch % 1 == 0:
        print(f'epoch {epoch}, EV = {ev}, val  loss = {loss} , train loss {sum(losses_train[-10:])/10}, pcc = {pcc}')
        #print(f'epoch {epoch}, EV = {ev}, val  loss = {loss} , train loss {sum(losses_train[-10:])/10}')
    if not_improve == endure:
        print("Early stopping!")

    scheduler.step()



C:\Users\admin\anaconda3\envs\neural_control\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/2000 [00:00<?, ?it/s]

epoch 0, EV = -120296.9453125, val  loss = 178516.615625 , train loss 253563.2328125, pcc = -0.011878948658704758


RuntimeError: CUDA out of memory. Tried to allocate 814.00 MiB (GPU 0; 8.00 GiB total capacity; 4.27 GiB already allocated; 532.98 MiB free; 5.68 GiB reserved in total by PyTorch)

In [ ]:
max(EVs)

In [ ]:
exp = "layer1_67"
torch.save(encoder, f'./exp{exp}.pt')